In [36]:
import pandas as pd
import numpy as np
import xarray as xr
import hvplot.pandas
import os.path as osp
from tqdm import tqdm
import seaborn as sns
import holoviews as hv

In [6]:
PRJ_DIR = '/data/SFIMJGC_HCP7T/2025_BCBL_VisitNIH'
sbj_list = ['sub-001',  'sub-002',  'sub-003',  'sub-004',  'sub-007',  'sub-008']
ses_list = ['ses-01',  'ses-02',  'ses-03',  'ses-04',  'ses-05',  'ses-06',  'ses-07',  'ses-08',  'ses-09',  'ses-10']
num_rest_ics = 20

In [22]:
df = pd.DataFrame(columns=['sbj','ses','rs_ic','bh_ic','R','p_val (param)','p_val (burton2020)'])
df_list = []
for sbj in sbj_list:
    for ses in tqdm(ses_list):
        for rs_ic in range(num_rest_ics):
            input_path = osp.join(PRJ_DIR,'data','prcs_data',sbj,'burton2020',f'{sbj}_REST-IC-{rs_ic}_vs_BH_{ses}.txt')
            aux = pd.read_csv(input_path, index_col=[0])
            num_bhs = aux.shape[0]
            for bh_ic in range(num_bhs):
                df_list.append({'sbj':sbj, 'ses':ses, 'rs_ic':rs_ic, 'bh_ic':bh_ic, 'R':aux.loc[bh_ic,'R'],'p_val (param)':aux.loc[bh_ic,'p_val (parametric)'], 'p_val (burton2020)':aux.loc[bh_ic,'p_val (non parametric)']})

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 20.17it/s]


In [23]:
df = pd.DataFrame(df_list)

In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62540 entries, 0 to 62539
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   sbj                 62540 non-null  object 
 1   ses                 62540 non-null  object 
 2   rs_ic               62540 non-null  int64  
 3   bh_ic               62540 non-null  int64  
 4   R                   62540 non-null  float64
 5   p_val (param)       62540 non-null  float64
 6   p_val (burton2020)  62540 non-null  float64
dtypes: float64(3), int64(2), object(2)
memory usage: 3.3+ MB


In [82]:
sbj       = 'sub-007'
num_comps = df.set_index(['sbj']).loc[sbj].shape[0]
print(sbj,num_comps)

sub-007 8680


In [83]:
df_significat = df[df['p_val (burton2020)']< 0.05 / num_comps ]

In [84]:
df.set_index(['sbj','ses']).loc[sbj].apply(np.abs).hvplot.violin(by='rs_ic',y='R', width=1500) * \
df.set_index(['sbj','ses']).loc[sbj].apply(np.abs).hvplot.scatter(x='rs_ic',y='R', size=5, hover_cols=['sbj','ses','p_val (burton2020)','bh_ic']) * \
df_significat.set_index(['sbj','ses']).loc[sbj].apply(np.abs).hvplot.scatter(x='rs_ic',y='R', c='r',hover_cols=['sbj','ses','p_val (burton2020)','bh_ic'], size=5)

:Overlay
   .Violin.I   :Violin   [rs_ic]   (R)
   .Scatter.I  :Scatter   [rs_ic]   (R,ses,p_val (burton2020),bh_ic)
   .Scatter.II :Scatter   [rs_ic]   (R,ses,p_val (burton2020),bh_ic)